# Catchement averaging

In [11]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
import pcraster as pcr
import numpy as np
from netCDF4 import Dataset
import rioxarray
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
import time
from multiprocess import Pool
import tqdm

In [12]:
# set your working directory
directoryMain = '/Users/niekcollotdescury/Desktop/Applied data science/Thesis/code/data/satellite_data/'
directoryArea = '/Users/niekcollotdescury/Desktop/Applied data science/Thesis/code/data/satellite_data/'

In [13]:
# change working directory
os.chdir(directoryMain)

## Transormations cell area and ldd maps

In [19]:
# get grid area: all products should have same grid
os.system('cdo gridarea lwe_thickness_GRACE_data_meter.nc area/gridarea30min.nc')

0

In [7]:
# transform to tiff
os.system('gdal_translate area/gridarea30min.nc  area/gridarea30min.tif')
os.system('gdal_translate area/lddsound_30min.nc area/lddsound_30min.tif')

# transform from tiff to map (pcraster format)
os.system('gdal_translate -of PCRaster area/gridarea30min.tif  area/gridarea30min.map')
os.system('gdal_translate -of PCRaster area/lddsound_30min.tif area/lddsound_30min.map')
os.system('rm *.xml') # remove redundant files

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.


rm: *.xml: No such file or directory


256

In [8]:
# change directory for ldd calculation
os.chdir(directoryArea)
# transform ldd map to .ldd format
os.system('pcrcalc lddsound_30min.ldd = "lddrepair(ldd(lddsound_30min.map))"')
# aguila lddsound_30min.ldd --> command in terminal for viewing the map

pcrcalc version: 4.4.0 (darwin/x86_64)


0

In [9]:
# reset working directory
os.chdir(directoryMain)

In [14]:
# define cell area file and ldd file
cell_area_file = "area/gridarea30min.map"
ldd_file       = "area/lddsound_30min.ldd"

# calculate catchment area
# - set clone, the bounding box of your study area - here, we use ldd 
clone_file     = ldd_file
pcr.setclone(clone_file)
# - read cell_area and ldd files
cell_area = pcr.readmap(cell_area_file)
ldd       = pcr.readmap(ldd_file)

# - calculate catchment area
catchment_area = pcr.catchmenttotal(cell_area, ldd)
# - save catchment_area to a file - note the file output will be under the work_dir
catchment_area_file = "area/catchment_area.map"

In [21]:
pcr.report(catchment_area, catchment_area_file)

In [15]:
# function for checking if directory exists and otherwise creating it
def check_dir_or_make(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(path)


In [16]:
def create_single_timestep_maps(var, input_file):
    check_dir_or_make(var)
    check_dir_or_make(f'{var}/timesteps')
    
    xds = rioxarray.open_rasterio(input_file)
    time = pd.DataFrame(xds[:,0,0].time.to_numpy(), columns=['date'])
    time['date'] = time['date'].astype('str')
    time[['year', 'month', 'left']]= time.date.str.split('-', expand = True)

    for i in range(len(xds[:,0,0])):
        timestep = time.iloc[i,:]
        name = f'{timestep.year}_{timestep.month}'
        print(i+1)
        print(name)
        
        single = f'{var}/timesteps/{var}_{name}'

        cmd = f'cdo seltimestep,{i+1} {input_file} {single}.nc'
        os.system(cmd)

        cmd = f'gdal_translate {single}.nc {single}.tif'
        os.system(cmd)

        cmd = f"gdal_translate -of PCRaster {single}.tif {single}.map"
        os.system(cmd)
    
    os.system(f'rm {var}/timesteps/*.xml')
    os.system(f'rm {var}/timesteps/*.tif')
    os.system(f'rm {var}/timesteps/*.nc')


In [ ]:
satelliteProducts = ['lwe', 'sc', 'sm']
inputFiles     = ["lwe_thickness_GRACE_data_meter.nc",
                  "GHRM5C_1978-2021_float32_masked_monthly_30min.nc",
                  "monmean_sm_ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED_1978-2019_half_degree.nc"]

for sp in range(len(satelliteProducts)):
    create_single_timestep_maps(satelliteProducts[sp],
                               inputFiles[sp])

#### Possible check if the map is correct

In [57]:
# - make sure that the file has the same map attributes as the ldd file
cmd = f"mapattr -c lddsound_30min.ldd {input_file}.map"
os.system(cmd)

mapattr version: 4.4.0 (darwin/x86_64)


0

## Calculate upstream averaged values for catchment areas

In [17]:
%%time
def upstream_maps(input_files):
    
    sat = input_files.split('/')[0]        
    name = input_files.split('/')[-1]

    
    
    
    cell_value = pcr.readmap(f"{input_files}")

    # check if cell value is present with boolean map
    cell_value_defined = pcr.defined(cell_value)
    
    # for missing values, set cell_value to zero, and cell_area (weight) to zero 
    cell_value_covered = pcr.ifthenelse(cell_value_defined, cell_value, pcr.scalar(0.0))
    cell_area_weight   = pcr.ifthenelse(cell_value_defined, cell_area, pcr.scalar(0.0))
    
    # catchment area (NOTE: this is based on cell_area_weight)
    catchment_area = pcr.catchmenttotal(cell_area_weight, ldd) 
    
    # calculate upstream/catchment average values (NOTE: this uses cell_value_covered and  cell_area_weight)
    upstream_average_value = pcr.catchmenttotal(cell_value_covered * cell_area_weight, ldd) / catchment_area
    pcr.report(upstream_average_value, f"{sat}/upstream/upstream_{name}")

    file_in = f'{sat}/upstream/upstream_{name}'
    file_out = file_in.replace('.map', '.nc')

    cmd = f'pcr2nc -i {file_in} -o {file_out} -m {sat}/nc_metadata_1.yaml'
    os.system(cmd)
        

CPU times: user 15 µs, sys: 1 µs, total: 16 µs
Wall time: 22.6 µs


In [21]:
def parralel_runs(var):
    print(var)
    input_files = glob.glob(f'{var}/timesteps/*.map')


    check_dir_or_make(f'{var}/upstream')
    cmd = f'rm {var}/upstream/*'
    os.system(cmd)

    time.sleep(4)

    pool = Pool(processes=36) # set number of cores

    for _ in tqdm.tqdm(pool.imap_unordered(upstream_maps, input_files), total=len(input_files)):
        pass
# run code for all satellite products
# add correct yaml file before execution to the satellite folder 
# parralel_runs('sm')
# parralel_runs('sc')
parralel_runs('lwe')

lwe


rm: lwe/upstream/*: No such file or directory
  0%|                                                   | 0/181 [00:00<?, ?it/s][11:26:47] - Defining WGS84 coordinates variables
[11:26:47] - Applying compression level 9
[11:26:47] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[11:26:47] - Defining WGS84 coordinates variables
[11:26:47] - Applying compression level 9
[11:26:47] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[11:26:47] - Adding lwe/upstream/upstream_lwe_2004_03.map - timestep None - hour 0.0
[11:26:47] - Writing lwe/upstream/upstream_lwe_2004_03.nc
[11:26:47] - Adding lwe/upstream/upstream_lwe_2020_02.map - timestep None - hour 0.0
[11:26:47] - Writing lwe/upstream/upstream_lwe_2020_02.nc
[11:26:48] - Defining WGS84 coordinates variables
[11:26:48] - Applying compression level 9
[11:26:48] - Creating NETCDF3_CLASSIC main variable lwe: type floa

In [ ]:
# example yaml file --> depends on the netcdf file information for the satellite data

# variable:
#   shortname: lwe
#   description: Lwe_thickness
#   longname: lwe_thickness
#   units: m
#   compression: 9
#   least_significant_digit: 2
# source: JRC Space, Security, Migration
# reference: JRC Space, Security, Migration
# geographical:
#   datum: WGS84
# time:
#   calendar: proleptic_gregorian
#   units: days since 1996-01-01
